# Rates rebate legislation as code

Using the documentation [here](https://docs.google.com/document/d/1b4RxWRqXb65X2POi4NMRMMrSMJQ3bgKO9SgRGjJev8E/edit) as a basis for implementing literate python for the implementation of the rates rebate calculation.



### _Relevant date_ means:

The commencement of the rating year in respect of which an application for a  subsidy under this Act is made
The date at the commencement of the rating year is 1 July of the rating year.



In [105]:
from datetime import date
# TODO fix bug. if the date is 23rd feb 2018 then this should return 1 july 2017
relevant_date = date(date.today().year, 7, 1)

### _Residential Property_ means:

A rating unit under the Local Government (Rating) Act 2002 that is not used principally for commercial or industrial or business or farming purposes


In [106]:
is_residential_property = True

### Subsidy means:

1. A rates subsidy. See the [application definition](rates_rebate_as_plain_old_python.ipynb#Application-for-a-rates-subsidy-means)
2. A retirement village subsidy. See the [application definition](rates_rebate_as_plain_old_python.ipynb#Application-for-a-retirement-village-subsidy-means)

In [107]:
# In code we represent these two possible outcomes as a boolean.
# Lesson learned: if the policy intent is to provide a subsidy 
# with additional methods, a more expressive type will need to be used.
is_retirement_subsidy = False

### _Ratepayer_ means:

At the _relevant date_

1. A person is the ratepayer of a property if the person appears as the ratepayer on the rating register of the council the person applies to.

**OR**

2. A person is the ratepayer of a property if the person lives in a retirement home.

**TODO** refering to a retirement village as a ratepayer doesn't make sense anymore

In [108]:
is_ratepayer = True
is_retirement_vilage_resident = False


### Usual place of residence
At the [relevant date](rates_rebate_as_plain_old_python.ipynb#Relevant-date-means:) the person is living at the residential property which they are claiming rates for

In [109]:
usual_place_of_residence = True

### _Application for a rates subsidy_ means

A person may apply for a rates subsidy if, on the _relevant date_:

1. the person is a ratepayer; and
2. the property for which the rates are paid is a residential property; and
3. the property is the usual place of residence of the ratepayer.


In [122]:
is_eligible = False
if is_ratepayer and is_residential_property and usual_place_of_residence:
    is_retirement_subsidy = False
    is_eligible = True


### _Application for a retirement village subsidy_ means

A person may apply for a retirement village subsidy if, on the relevant date, the person -

1. is a resident of a retirement village; and
2. has a residential unit in the retirement village that is not separately rated; and
3. contributes to the outgoings of the retirement village.


In [124]:
has_residential_unit = False #todo define/move elsewhere
if is_retirement_vilage_resident and has_residential_unit and contributes_to_outgoings_of_retirement_village:
    is_eligible = True
    is_retirement_subsidy = True
    

### _Number of dependants_ means

1. a child who, on the relevant date, -
```
    a. is under 18 years of age; and
    b. ordinately resides at the property; and
    c. etc
```
2. a person who, on the relevant date, - 
```
    a. is related by blood, marriage, etc; and
    b. receives a social security benefit under the Social Security Act 1964; and 
    c. is ordinarily resident at the property; and
    d. is not a child, spouse or partner of the ratepayer or resident (as applicable)
```

**Technical notes**

- represent this as a number, for the purpose of our demonstration we felt this detail.
- the number of depandants is determined by local government and supplied via data access layer to DIA

In [120]:
number_of_dependants = 0

### _Payment made_ means

1. for a ratepayer, the amount of the rates payable for the year.

2. for a resident, either-

    a. that part of the resident’s contribution to the outgoings of the retirement village that is made specifically towards the rates payable in respect of the retirement village; or

    b. if no part of the resident’s annual contribution to the outgoings of the retirement village is specifically made in respect of rates, then the resident is deemed to have contributed in respect of rates an amount that bears to the total amount of rates payable in respect of the retirement village the same proportion as the amount of the resident’s contribution bears to the total amount of the contributions of all the residents of the retirement village


**Technical notes**

1. For retrieving the amount of rates, it makes senes for this to come from a data access layer provided by local government organizations.
2. For a resident of a retirement village, the data access layer should be provided by MBIE as per retirements villages act 2003.

In [126]:
# TODO: check to see if retirment_resident to determine where data comes from
if is_retirement_subsidy:
#     fetch from MBIE
    pass
else:
#     fetch from local govt
    pass
amount_of_rates = 4500

### _Income_ means:

1. for a ratepayer, [to be taken from the definition of income in section 2(1) of the Rates Rebate Act 1973]
2. for a resident of a retirement village, [to be taken from the definition of income in section 2(1) of the Rates Rebate Act 1973, but amended as set out in section 7A(3) of that Act]

**Technical Notes**

1. for a ratepayer, data is the responsibility of local government
2. for a resident of a retirement village data is the responsibility of MBIE


In [131]:
household_income = 45000

### _Rates calculation_ means:


The amount of subsidy a person is entitled to is calculated in accordance with the following formula:

```
Rates subsidy = household income - baserate / dependents.number * 10% * rates
```

**Technical notes**
- Baserate is a constant 15,000
- This is an approximation of the subsidy calculation used for the purpose of the 'better rules' investigation.
- In the technical model this calculation would be provided by DIA
- In the technical model this would be called by local govenment
- In the technical model the calculation result would need verification from the local government so that funds can be released from central government to local government

In [135]:
base_rate = 15000
def rate_subsidy_calculation(household_income, dependants_number, rate):
    # don't divide by 0
    rate = max(1, rate)
    dependants_number = max(1, dependants_number) 

    return (household_income - base_rate) / (dependants_number * 0.1 * rate)


### Therefore

Given the above we can calculate:

1. Your entitlement amount
2. Your method of entitlement, either 'rates subsidy' or 'retirement subsidy'


In [134]:
if is_eligible:
    outcome_method = 'rates subsidy'
    if is_retirement_subsidy:
        outcome_method = 'retirement subsidy'

    outcome_entitlement = rate_subsidy_calculation(household_income, number_of_dependants, amount_of_rates)
    print('You are entitled to a {}: ${:.2f}'.format(outcome_method, outcome_entitlement))
else:
    print("You are not eligible for the rates rebate subsidy")


You are entitled to a rates subsidy: $66.67


# Ideas/TODO

- Test cases
- make content dynamic (I.E insert relevant_date)
- Make the question model executable
- Make a flow diagram output from the digital ruleset